In [1]:
from transformers import pipeline
from collections import defaultdict

# Replace this with your own checkpoint
# model_checkpoint = "/home/tanluuuuuuu/Desktop/luunvt/direct_indirect/models/model_from_2023-11-09/roberta-base_20:53:28/best_f1"
model_checkpoint = "/home/tanluuuuuuu/Desktop/luunvt/direct_indirect/models/model_from_2023-11-24/roberta-base_11:29:56/best_f1"
token_classifier = pipeline(
    "ner", model=model_checkpoint, aggregation_strategy="simple", device=0
)

2023-11-30 10:13:26.017608: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-30 10:13:26.043832: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-30 10:13:26.445033: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def preprocess_description(description, words_need_removed = []):
    # add space to string
    single_description = description.strip()
    new_description = []
    last_special = -1
    for idx, letter in enumerate(single_description):
        if not (('a' <= letter and letter <= 'z') or ('A' <= letter and letter <= 'Z') or ('0' <= letter and letter <= '9') or letter == ' '):
            pretext = single_description[last_special + 1:idx].strip()
            if pretext != '' and pretext != ' ':
                new_description.append(pretext)
            new_description.append(letter.strip())
            last_special = idx
        if idx == len(single_description) - 1:
            new_description.append(
                single_description[last_special + 1:idx + 1].strip())
    new_description = " ".join(new_description)
    
    # Remove words from string: brand name,...
    querywords = new_description.split(" ")
    words_need_removed = [x.lower() for x in words_need_removed]
    resultwords  = [word for word in querywords if word.lower() not in words_need_removed]
    result = ' '.join(resultwords)
    
    return result

In [3]:
import json

# Replace this description
description = '''
Product 1: Kettlebell supports a wide range of resistance-training exercises
Product 1: Made of solid high-quality cast iron for reliable built-to-last strength
Product 1: Painted surface for increased durability and corrosion protection
Product 1: Textured wide handle helps ensure a comfortable, secure grip; hold with one hand or two
Product 2: Kettlebell supports a wide range of resistance-training exercises
Product 2: Made of solid high-quality cast iron for reliable built-to-last strength
Product 2: Painted surface for increased durability and corrosion protection
Product 2: Textured wide handle helps ensure a comfortable, secure grip; hold with one hand or two
'''

high_score_ans = defaultdict(set)
bullet_points = description.split("\n")
for bullet_point in bullet_points:
    bullet_point = preprocess_description(bullet_point)

    if bullet_point != "":
        print(bullet_point)

        results = token_classifier(bullet_point)
        for res in results:
            if res['word'].lower().strip() in ['durable', 'strong', 'heavy-duty', 'heavy duty', 'stability', 'versatile', 'comfortable']:
                continue
            group = res['entity_group']
            if res['score'] >= 0.9:
                high_score_ans[group].add(res['word'].lower().strip() + " " + str(res['score']))
                
new_high_score_ans = defaultdict(list)
for key_dict in high_score_ans.keys():
    new_high_score_ans[key_dict] = list(high_score_ans[key_dict])

print("-"*100)            
print(json.dumps(new_high_score_ans, sort_keys=True, indent=4))                

Product 1 : Kettlebell supports a wide range of resistance - training exercises
Product 1 : Made of solid high - quality cast iron for reliable built - to - last strength
Product 1 : Painted surface for increased durability and corrosion protection
Product 1 : Textured wide handle helps ensure a comfortable , secure grip ; hold with one hand or two
Product 2 : Kettlebell supports a wide range of resistance - training exercises
Product 2 : Made of solid high - quality cast iron for reliable built - to - last strength
Product 2 : Painted surface for increased durability and corrosion protection
Product 2 : Textured wide handle helps ensure a comfortable , secure grip ; hold with one hand or two
----------------------------------------------------------------------------------------------------
{
    "MAT": [
        "cast iron 0.99997795"
    ]
}


In [40]:
import json

# Replace this description
description = '''
♥️ Rattan Material: PE
'''

high_score_ans = defaultdict(set)
bullet_points = description.split("\n")
for bullet_point in bullet_points:
    bullet_point = preprocess_description(bullet_point)

    if bullet_point != "":
        print(bullet_point)

        results = token_classifier(bullet_point)
        for res in results:
            if res['word'].lower().strip() in ['durable', 'strong', 'heavy-duty', 'heavy duty', 'stability', 'versatile', 'comfortable']:
                continue
            group = res['entity_group']
            if res['score'] >= 0.99:
                # high_score_ans[group].add(res['word'].lower().strip() + " " + str(res['score']))
                high_score_ans[group].add(res['word'].lower().strip())
                
new_high_score_ans = defaultdict(list)
for key_dict in high_score_ans.keys():
    new_high_score_ans[key_dict] = list(high_score_ans[key_dict])

print("-"*100)            
print(json.dumps(new_high_score_ans, sort_keys=True, indent=4))                

♥ ️ Rattan Material : PE
----------------------------------------------------------------------------------------------------
{
    "MAT": [
        "rattan",
        "pe"
    ]
}


In [31]:
import json
from spacy import displacy

# Replace this description
description = '''
Product 1 : MIDNIGHT - BLUE DRESSER FOR BEDROOM : This chest of bedroom drawers measures 17 . 52 " L x 35 . 24 " W x 30 . 82 " H and weighs 65 . 56 lbs . The compact size of the handy drawer dresser helps you maximize your space while creating a place for clothes , books , games , or trinkets needing a home .
'''

ner_tags = [
"MAT",
"NMAT",
"DIMENSION",
"WEIGHT",
"TARGET_USER",
"PROPERTY",
"COLOR",
"SHAPE",
"SIZE",
]

colors = {
    "MAT": "#EEE",
    "NMAT": "#FFD",
    "DIMENSION": "#EFD",
    "WEIGHT": "#173",
    "TARGET_USER": "#379",
    "PROPERTY": "#FCF",
    "COLOR": "#FEF",
    "SHAPE": "#C9C",
    "SIZE": "#B8E"
         }
options = {"ents": ner_tags, "colors": colors}

high_score_ans = defaultdict(set)
bullet_points = description.split("\n")
for bullet_point in bullet_points:
    # bullet_point = preprocess_description(bullet_point)

    if bullet_point != "":
        results = token_classifier(bullet_point)
        list_ents = []
        for otp in results:
            if otp['score'] < 0.9:
                continue
            list_ents.append({
                'label': otp['entity_group'],
                'start': otp['start'],
                'end': otp['end'],
            })
        doc_manual = {
            'text': bullet_point,
            'ents': list_ents
        }
        displacy.render(
            doc_manual,
            style='ent',
            manual=True,
            options=options
        )              

In [7]:
import json
from spacy import displacy

# Replace this description
description = '''
🐶 【 Ultra - Soft Dog beds for Large Dogs 】 The soft dog bed for crate is made of fluffy fluff with a swirl design , which can provide good support while relieving dog joint and muscle pain . The large dog bed has a full padding and the fluffy dog crate pad crate bed is very cozy and warm to create a healthy sleeping environment for pet dogs / cats to curl up and help them calm down .
'''

ner_tags = [
"MAT",
"NMAT",
"DIMENSION",
"WEIGHT",
"TARGET_USER",
"PROPERTY",
"COLOR",
"SHAPE",
"SIZE",
]

colors = {
    "MAT": "#EEE",
    "NMAT": "#FFD",
    "DIMENSION": "#EFD",
    "WEIGHT": "#173",
    "TARGET_USER": "#379",
    "PROPERTY": "#FCF",
    "COLOR": "#FEF",
    "SHAPE": "#C9C",
    "SIZE": "#B8E"
         }
options = {"ents": ner_tags, "colors": colors}

high_score_ans = defaultdict(set)
bullet_points = description.split("\n")
for bullet_point in bullet_points:
    # bullet_point = preprocess_description(bullet_point)

    if bullet_point != "":
        results = token_classifier(bullet_point)
        display(results)
        list_ents = []
        for otp in results:
            if otp['score'] < 0.9:
                continue
            list_ents.append({
                'label': otp['entity_group'],
                'start': otp['start'],
                'end': otp['end'],
            })
        doc_manual = {
            'text': bullet_point,
            'ents': list_ents
        }
        displacy.render(
            doc_manual,
            style='ent',
            manual=True,
            options=options
        )
        print()

[]